# What

As part of the produciton challenge #172. I need to be able to upload the results to an external server.


## Modules

In [ ]:
# Local

# Third Party
import lancedb
import pandas as pd

# Built in


# Upload output to viewer vector_db
The end goal will be to take the embeddings datasets and put them into a vector database that is in the cloud.

However right not I am not working on it so I can just create this local script that will take the embedding folder and move it into a vector_db database within the viewer.

In [ ]:
db = lancedb.connect("../../viewer/vector_db")
db.table_names()

In [ ]:
important_text_embeddings = pd.read_pickle("../../output/embeddings/important_text_embeddings.pkl")

important_text_embeddings.rename(columns={'important_text_embedding': 'vector'}, inplace=True)
db.create_table("important_text_embeddings", important_text_embeddings, mode="overwrite")


In [ ]:
recommendations_embeddings = pd.read_pickle("../../output/embeddings/recommendations_embeddings.pkl")

recommendations_embeddings.rename(columns={'recommendation_embedding': 'vector'}, inplace=True)
db.create_table("recommendation_embeddings", recommendations_embeddings, mode="overwrite")

In [ ]:
report_sections_embeddings = pd.read_pickle("../../output/embeddings/report_sections_embeddings.pkl")

report_sections_embeddings.rename(columns={'section_embedding': 'vector'}, inplace=True)
section_table = db.create_table("report_section_embeddings", report_sections_embeddings, mode="overwrite")

section_table.create_fts_index("section_text", replace=True)

In [ ]:
safety_issues_embeddings = pd.read_pickle("../../output/embeddings/safety_issues_embeddings.pkl")

safety_issues_embeddings.rename(columns={'safety_issue_embedding': 'vector'}, inplace=True)
si_table = db.create_table("safety_issue_embeddings", safety_issues_embeddings, mode="overwrite")

si_table.to_pandas()

### Get recomemdnations linked to safety issues

In [ ]:
rec_si_links = pd.read_pickle("../../output/recommendation_safety_issue_links.pkl")
rec_si_links

In [ ]:
rec_si_links_long = pd.concat(
    rec_si_links['recommendation_links'].tolist(),
    ignore_index=True
)
rec_si_links_long = rec_si_links_long.query('link == "Confirmed"')
rec_si_links_long

In [ ]:
# Squish it so that it ijust a list of safety issue ids with the recommendations

grouped_df = rec_si_links_long.groupby('safety_issue_id').apply(
    lambda x: x[['recommendation_id', 'recommendation', 'recipient']].reset_index(drop=True)
).reset_index().drop('level_1', axis=1)

grouped_df

In [ ]:
safety_issue_recommendations_table = db.create_table(
    'safety_issue_recommendations',
    grouped_df,
    mode = "overwrite"
)

safety_issue_recommendations_table.search().where("safety_issue_id = '2000_004_2'").to_pandas()

## Creating test vector_db

In [ ]:
uri = '../../tests/data/vector_db'

test_db = lancedb.connect(uri)

In [ ]:
si_table = test_db.create_table(
    'safety_issue_embeddings',
    safety_issues_embeddings.sample(frac=0.1, random_state = 42),
    mode = "overwrite"
)

si_table.create_fts_index('safety_issue', replace=True)

si_table.to_pandas()

In [ ]:
sections_table = test_db.create_table(
    'report_section_embeddings',
    section_text_embeddings.sample(frac=0.1, random_state = 42),
    mode = "overwrite"  
)
sections_table.create_fts_index('section_text', replace=True)

sections_table.to_pandas()

In [ ]:
safety_issue_recommendations_test_table = test_db.create_table(
    'safety_issue_recommendations',
    grouped_df.sample(frac=0.1, random_state = 42),
    mode = "overwrite"
)

safety_issue_recommendations_test_table.to_pandas().query('safety_issue_id == "2002_007_1"')